<a href="https://colab.research.google.com/github/whitebear909/rl/blob/master/A3C_OneNet_Dropout_t_20190829.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = ""

import threading
import gym
import time
import multiprocessing
import numpy as np
from queue import Queue
import argparse
import matplotlib.pyplot as plt


import tensorflow as tf
from tensorflow.python import keras
from tensorflow.python.keras import layers



In [0]:
from google.colab import auth
auth.authenticate_user()
from google.colab import drive
drive.mount('/content/gdrive')

In [0]:
tf.enable_eager_execution()

In [0]:

# parser = argparse.ArgumentParser(description='Run A3C algorithm on the game '
#                                              'Cartpole.')
# parser.add_argument('--algorithm', default='a3c', type=str,
#                     help='Choose between \'a3c\' and \'random\'.')
# parser.add_argument('--train', dest='train', action='store_true',
#                     help='Train our model.')
# parser.add_argument('--lr', default=0.001,
#                     help='Learning rate for the shared optimizer.')
# parser.add_argument('--update-freq', default=20, type=int,
#                     help='How often to update the global model.')
# parser.add_argument('--max-eps', default=1000, type=int,
#                     help='Global maximum number of episodes to run.')
# parser.add_argument('--gamma', default=0.99,
#                     help='Discount factor of rewards.')
# parser.add_argument('--save-dir', default='/tmp/', type=str,
#                     help='Directory in which you desire to save the model.')

# args = parser.parse_args()



usage: ipykernel_launcher.py [-h] [--algorithm ALGORITHM] [--train] [--lr LR]
                             [--update-freq UPDATE_FREQ] [--max-eps MAX_EPS]
                             [--gamma GAMMA] [--save-dir SAVE_DIR]
ipykernel_launcher.py: error: unrecognized arguments: -f /root/.local/share/jupyter/runtime/kernel-719f13bd-1935-4bf5-9219-250fe07683a8.json


SystemExit: ignored

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2890: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [0]:
import easydict

args = easydict.EasyDict({
 
        "algorithm": "a3c",
        
        "game_name": "CartPole-v1",
 
        "train": "train",
 
        "lr": 0.01,
    
        "update_freq": 20 ,

        "max_eps": 3000 ,

        "gamma": 0.99,
 
        "save_dir": "gdrive/My Drive/Deep Learning/A3C"

})


Episode: 241 | Moving Average Reward: 146 | Episode Reward: 16 | Loss: 2.003 | Steps: 17 | Worker: 2Episode: 241 | Moving Average Reward: 146 | Episode Reward: 17 | Loss: 5.392 | Steps: 18 | Worker: 0

Episode: 243 | Moving Average Reward: 145 | Episode Reward: 17 | Loss: 2.299 | Steps: 18 | Worker: 3
Episode: 244 | Moving Average Reward: 144 | Episode Reward: 18 | Loss: 2.417 | Steps: 19 | Worker: 0


In [0]:
class ActorCriticModel(keras.Model):
  def __init__(self, state_size, action_size):
    super(ActorCriticModel, self).__init__()
    self.state_size = state_size
    self.action_size = action_size
    self.dense1 = layers.Dense(128, activation='relu', kernel_initializer = 'he_uniform')
    self.dense2 = layers.Dense(128, activation='relu', kernel_initializer = 'he_uniform')
    self.dropout = layers.Dropout(0.25)
    self.policy_logits = layers.Dense(action_size, kernel_initializer = 'he_uniform')    
    self.values = layers.Dense(1, kernel_initializer = 'he_uniform')

  def call(self, inputs):
    # Forward pass
    # x = self.dense1(inputs)
    # logits = self.policy_logits(x)
    # v1 = self.dense2(inputs)
    # values = self.values(v1)
    # ver 1.1
    x = self.dense1(inputs)
    x = self.dropout(x)
    x = self.dense2(x)
    x = self.dropout(x)
        
    logits = self.policy_logits(x)
    values = self.values(x)


    return logits, values

def record(episode,
           episode_reward,
           worker_idx,
           global_ep_reward,
           result_queue,
           total_loss,
           num_steps):
  """Helper function to store score and print statistics.
  Arguments:
    episode: Current episode
    episode_reward: Reward accumulated over the current episode
    worker_idx: Which thread (worker)
    global_ep_reward: The moving average of the global reward
    result_queue: Queue storing the moving average of the scores
    total_loss: The total loss accumualted over the current episode
    num_steps: The number of steps the episode took to complete
  """
  if global_ep_reward == 0:
    global_ep_reward = episode_reward
  else:
    global_ep_reward = global_ep_reward * 0.99 + episode_reward * 0.01
  print(
      f"Episode: {episode} | "
      f"Moving Average Reward: {int(global_ep_reward)} | "
      f"Episode Reward: {int(episode_reward)} | "
      f"Loss: {int(total_loss / float(num_steps) * 1000) / 1000} | "
      f"Steps: {num_steps} | "
      f"Worker: {worker_idx}"
  )
  result_queue.put(global_ep_reward)
  return global_ep_reward



class MasterAgent():
  def __init__(self):
    self.game_name = args.game_name
    save_dir = args.save_dir
    self.save_dir = save_dir
    if not os.path.exists(save_dir):
      os.makedirs(save_dir)

    env = gym.make(self.game_name)
    self.state_size = env.observation_space.shape[0]
    self.action_size = env.action_space.n
    self.opt = tf.train.AdamOptimizer(args.lr, use_locking=True)
    print(self.state_size, self.action_size)

    self.global_model = ActorCriticModel(self.state_size, self.action_size)  # global network
    self.global_model(tf.convert_to_tensor(np.random.random((1, self.state_size)), dtype=tf.float32))

  def train(self):
    if args.algorithm == 'random':
      random_agent = RandomAgent(self.game_name, args.max_eps)
      random_agent.run()
      return

    res_queue = Queue()

    workers = [Worker(self.state_size,
                      self.action_size,
                      self.global_model,
                      self.opt, res_queue,
                      i, game_name=self.game_name,
                      save_dir=self.save_dir) for i in range(multiprocessing.cpu_count()*2)]

    for i, worker in enumerate(workers):
      print("Starting worker {}".format(i))
      time.sleep(1)
      worker.start()

    moving_average_rewards = []  # record episode reward to plot

    while True:
      reward = res_queue.get()
      if reward is not None:
        moving_average_rewards.append(reward)
      else:
        break
    [w.join() for w in workers]

    plt.plot(moving_average_rewards)
    plt.ylabel('Moving average ep reward')
    plt.xlabel('Step')
    plt.savefig(os.path.join(self.save_dir,
                             '{} Moving Average.png'.format(self.game_name)))
    plt.show()

  def play(self):
    env = gym.make(self.game_name).unwrapped
    state = env.reset()
    model = self.global_model
    model_path = os.path.join(self.save_dir, 'model_{}.h5'.format(self.game_name))
    print('Loading model from: {}'.format(model_path))
    model.load_weights(model_path)
    done = False
    step_counter = 0
    reward_sum = 0

    try:
      while not done:
        env.render(mode='rgb_array')
        policy, value = model(tf.convert_to_tensor(state[None, :], dtype=tf.float32))
        policy = tf.nn.softmax(policy)
        action = np.argmax(policy)
        state, reward, done, _ = env.step(action)
        reward_sum += reward
        print("{}. Reward: {}, action: {}".format(step_counter, reward_sum, action))
        step_counter += 1
    except KeyboardInterrupt:
      print("Received Keyboard Interrupt. Shutting down.")
    finally:
      env.close()


class Memory:
  def __init__(self):
    self.states = []
    self.actions = []
    self.rewards = []

  def store(self, state, action, reward):
    self.states.append(state)
    self.actions.append(action)
    self.rewards.append(reward)

  def clear(self):
    self.states = []
    self.actions = []
    self.rewards = []


class Worker(threading.Thread):
  # Set up global variables across different threads
  global_episode = 0
  # Moving average reward
  global_moving_average_reward = 0
  best_score = 0
  save_lock = threading.Lock()

  def __init__(self,
               state_size,
               action_size,
               global_model,
               opt,
               result_queue,
               idx,
               game_name,
               save_dir):
    super(Worker, self).__init__()
    self.state_size = state_size
    self.action_size = action_size
    self.result_queue = result_queue
    self.global_model = global_model
    self.opt = opt
    self.local_model = ActorCriticModel(self.state_size, self.action_size)
    self.worker_idx = idx
    self.game_name = game_name
    self.env = gym.make(self.game_name).unwrapped
    self.save_dir = save_dir
    self.ep_loss = 0.0

  def run(self):
    total_step = 1
    mem = Memory()
    while Worker.global_episode < args.max_eps:
      current_state = self.env.reset()
      mem.clear()
      ep_reward = 0.
      ep_steps = 0
      self.ep_loss = 0

      time_count = 0
      done = False
      while not done:
        logits, _ = self.local_model(
            tf.convert_to_tensor(current_state[None, :],
                                 dtype=tf.float32))
        probs = tf.nn.softmax(logits)

        action = np.random.choice(self.action_size, p=probs.numpy()[0])
        new_state, reward, done, _ = self.env.step(action)
        if done:
          reward = -1
        ep_reward += reward
        mem.store(current_state, action, reward)

        if time_count == args.update_freq or done:
          # Calculate gradient wrt to local model. We do so by tracking the
          # variables involved in computing the loss by using tf.GradientTape
          with tf.GradientTape() as tape:
            total_loss = self.compute_loss(done,
                                           new_state,
                                           mem,
                                           args.gamma)
          self.ep_loss += total_loss
          # Calculate local gradients
          grads = tape.gradient(total_loss, self.local_model.trainable_weights)
          # Push local gradients to global model
          self.opt.apply_gradients(zip(grads,
                                       self.global_model.trainable_weights))
          # Update local model with new weights
          self.local_model.set_weights(self.global_model.get_weights())

          mem.clear()
          time_count = 0

          if done:  # done and print information
            Worker.global_moving_average_reward = \
              record(Worker.global_episode, ep_reward, self.worker_idx,
                     Worker.global_moving_average_reward, self.result_queue,
                     self.ep_loss, ep_steps)
            # We must use a lock to save our model and to print to prevent data races.
            if ep_reward > Worker.best_score:
              with Worker.save_lock:
                print("Saving best model to {}, "
                      "episode score: {}".format(self.save_dir, ep_reward))
                self.global_model.save_weights(
                    os.path.join(self.save_dir,
                                 'model_{}.h5'.format(self.game_name))
                )
                Worker.best_score = ep_reward
            Worker.global_episode += 1
        ep_steps += 1

        time_count += 1
        current_state = new_state
        total_step += 1
    self.result_queue.put(None)

  def compute_loss(self,
                   done,
                   new_state,
                   memory,
                   gamma=0.99):
    if done:
      reward_sum = 0.  # terminal
    else:
      reward_sum = self.local_model(
          tf.convert_to_tensor(new_state[None, :],
                               dtype=tf.float32))[-1].numpy()[0]

    # Get discounted rewards
    discounted_rewards = []
    for reward in memory.rewards[::-1]:  # reverse buffer r
      reward_sum = reward + gamma * reward_sum
      discounted_rewards.append(reward_sum)
    discounted_rewards.reverse()

    logits, values = self.local_model(
        tf.convert_to_tensor(np.vstack(memory.states),
                             dtype=tf.float32))
    # Get our advantages
    advantage = tf.convert_to_tensor(np.array(discounted_rewards)[:, None],
                            dtype=tf.float32) - values
    # Value loss
    value_loss = advantage ** 2

    # Calculate our policy loss
    policy = tf.nn.softmax(logits)
    entropy = tf.nn.softmax_cross_entropy_with_logits_v2(labels=policy, logits=logits)

    policy_loss = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=memory.actions,
                                                                 logits=logits)
    policy_loss *= tf.stop_gradient(advantage)
    policy_loss -= 0.01 * entropy
    total_loss = tf.reduce_mean((0.5 * value_loss + policy_loss))
    return total_loss




Episode: 316 | Moving Average Reward: 88 | Episode Reward: 7 | Loss: 0.864 | Steps: 8 | Worker: 3
Episode: 0 | Moving Average Reward: 29 | Episode Reward: 29 | Loss: 12.465 | Steps: 30 | Worker: 3
Saving best model to gdrive/My Drive/Deep Learning/A3C, episode score: 29.0
Episode: 0 | Moving Average Reward: 29 | Episode Reward: 32 | Loss: 2.478 | Steps: 33 | Worker: 1
Episode: 0 | Moving Average Reward: 29 | Episode Reward: 34 | Loss: 1.731 | Steps: 35 | Worker: 3
Saving best model to gdrive/My Drive/Deep Learning/A3C, episode score: 32.0
Saving best model to gdrive/My Drive/Deep Learning/A3C, episode score: 34.0
Episode: 2 | Moving Average Reward: 28 | Episode Reward: 12 | Loss: 1.586 | Steps: 13 | Worker: 2
Episode: 4 | Moving Average Reward: 28 | Episode Reward: 7 | Loss: 0.372 | Steps: 8 | Worker: 1
Episode: 5 | Moving Average Reward: 28 | Episode Reward: 22 | Loss: 3.92 | Steps: 23 | Worker: 1
Episode: 6 | Moving Average Reward: 28 | Episode Reward: 65 | Loss: 4.227 | Steps: 66 | 

In [0]:

class RandomAgent:
  """Random Agent that will play the specified game
    Arguments:
      env_name: Name of the environment to be played
      max_eps: Maximum number of episodes to run agent for.
  """
  def __init__(self, env_name, max_eps):
    self.env = gym.make(env_name)
    self.max_episodes = max_eps
    self.global_moving_average_reward = 0
    self.res_queue = Queue()

  def run(self):
    reward_avg = 0
    for episode in range(self.max_episodes):
      done = False
      self.env.reset()
      reward_sum = 0.0
      steps = 0
      while not done:
        # Sample randomly from the action space and step
        _, reward, done, _ = self.env.step(self.env.action_space.sample())
        steps += 1
        reward_sum += reward
      # Record statistics
      self.global_moving_average_reward = record(episode,
                                                 reward_sum,
                                                 0,
                                                 self.global_moving_average_reward,
                                                 self.res_queue, 0, steps)

      reward_avg += reward_sum
    final_avg = reward_avg / float(self.max_episodes)
    print("Average score across {} episodes: {}".format(self.max_episodes, final_avg))
    return final_avg


In [0]:
if __name__ == '__main__':
  print(args)
  master = MasterAgent()
  if args.train:
    master.train()
  else:
    master.play()

Episode: 66 | Moving Average Reward: 26 | Episode Reward: 27 | Loss: 2.88 | Steps: 28 | Worker: 1
Episode: 67 | Moving Average Reward: 26 | Episode Reward: 12 | Loss: 0.703 | Steps: 13 | Worker: 0
{'algorithm': 'a3c', 'game_name': 'CartPole-v1', 'train': 'train', 'lr': 0.01, 'update_freq': 20, 'max_eps': 3000, 'gamma': 0.99, 'save_dir': 'gdrive/My Drive/Deep Learning/A3C'}
4 2
Episode: 68 | Moving Average Reward: 26 | Episode Reward: 12 | Loss: 1.518 | Steps: 13 | Worker: 3
Episode: 68 | Moving Average Reward: 26 | Episode Reward: 11 | Loss: 0.788 | Steps: 12 | Worker: 3
Episode: 70 | Moving Average Reward: 26 | Episode Reward: 23 | Loss: 9.615 | Steps: 24 | Worker: 0
Episode: 70 | Moving Average Reward: 26 | Episode Reward: 28 | Loss: 7.707 | Steps: 29 | Worker: 1Episode: 70 | Moving Average Reward: 26 | Episode Reward: 26 | Loss: 2.876 | Steps: 27 | Worker: 2

Episode: 73 | Moving Average Reward: 25 | Episode Reward: 9 | Loss: 0.548 | Steps: 10 | Worker: 0
Starting worker 0
Episode: 

In [0]:
multiprocessing.cpu_count()

2